In [1]:
# 라이브러리

In [1]:
pip install torch torchvision nibabel transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
pip install torch torchvision nibabel pandas transformers medpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 10.6 MB/s eta 0:00:00
  Created wheel for medpy: filename=MedPy-0.5.2-py3-none-any.whl size=224709 sha256=fc9a6f951643c297fbda551385742aaeaae82b1523fa844c2bd5e16ff56bbbdc
  Stored in directory: /root/.cache/pip/wheels/d4/33/ed/aaac5a347fb8d41679ca515b8f5c49dfdf49be15bdbb9a905d
Successfully built medpy


In [3]:
pip install torch

In [4]:
import os
import pandas as pd
import numpy as np
import nibabel as nib

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
import torch.nn.functional as F
from medpy.io import load

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import random

# 랜덤시드 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

SEED = 42
set_seed(SEED)


# 데이터

## 텍스트 데이터 및 전처리

In [7]:
file_path = '/content/drive/MyDrive/KUBIG/CV 세션 프로젝트/Processed_OASIS_MRI_Data.csv'
try:
    data_text = pd.read_csv(file_path)
    print("CSV 파일을 성공적으로 불러왔습니다.")
    print(data_text.head())
except FileNotFoundError:
    print(f"{file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인하세요.")

CSV 파일을 성공적으로 불러왔습니다.
              ID M/F  CDR  Age             Group  \
0  OAS1_0001_MR1   F  0.0   74       NonDemented   
1  OAS1_0002_MR1   F  0.0   55       NonDemented   
2  OAS1_0003_MR1   F  0.5   73  VeryMildDementia   
3  OAS1_0004_MR1   M  0.0   28       NonDemented   
4  OAS1_0005_MR1   M  0.0   18       NonDemented   

                                           CLIP_Text  
0  This is an MRI scan of a 74-year-old female wi...  
1  This is an MRI scan of a 55-year-old female wi...  
2  This is an MRI scan of a 73-year-old female wi...  
3  This is an MRI scan of a 28-year-old male with...  
4  This is an MRI scan of a 18-year-old male with...  


In [8]:
df = pd.DataFrame(data_text)[['ID', 'CLIP_Text', 'Group']]
df.head()

,ID,CLIP_Text,Group
0,OAS1_0001_MR1,This is an MRI scan of a 74-year-old female wi...,NonDemented
1,OAS1_0002_MR1,This is an MRI scan of a 55-year-old female wi...,NonDemented
2,OAS1_0003_MR1,This is an MRI scan of a 73-year-old female wi...,VeryMildDementia
3,OAS1_0004_MR1,This is an MRI scan of a 28-year-old male with...,NonDemented
4,OAS1_0005_MR1,This is an MRI scan of a 18-year-old male with...,NonDemented


In [9]:
df['CLIP_Text'] = df['CLIP_Text'].str.replace(r'(male.*)', 'male', regex=True)

In [10]:
df.head()

,ID,CLIP_Text,Group
0,OAS1_0001_MR1,This is an MRI scan of a 74-year-old female,NonDemented
1,OAS1_0002_MR1,This is an MRI scan of a 55-year-old female,NonDemented
2,OAS1_0003_MR1,This is an MRI scan of a 73-year-old female,VeryMildDementia
3,OAS1_0004_MR1,This is an MRI scan of a 28-year-old male,NonDemented
4,OAS1_0005_MR1,This is an MRI scan of a 18-year-old male,NonDemented


In [11]:
df['Group'].value_counts()

,count
Group,
NonDemented,336
VeryMildDementia,70
MildDementia,28
ModerateDementia,2


## 이미지 데이터

In [12]:
data_path = '/content/drive/MyDrive/KUBIG/CV 세션 프로젝트/oasis/OASIS'

# 경로 내의 모든 .nii 파일 읽기
nii_files = [f for f in os.listdir(data_path) if f.endswith('.nii')]

# nii 파일을 읽어들이는 함수
def load_nii_file(file_path):
    img = nib.load(file_path)
    data = img.get_fdata()
    return data

# 모든 .nii 파일 데이터 로드
nii_data_list = []
for nii_file in nii_files:
    file_path = os.path.join(data_path, nii_file)
    try:
        data = load_nii_file(file_path)
        nii_data_list.append(data)
        print(f"{nii_file} 파일을 성공적으로 불러왔습니다.")
    except Exception as e:
        print(f"{nii_file} 파일을 불러오는 중 오류 발생: {str(e)}")

OAS1_0005_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0014_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0004_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0002_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0010_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0001_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0006_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0012_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0009_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0013_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0007_MR1_mpr_n3_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0003_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0011_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0017_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔습니다.
OAS1_0030_MR1_mpr_n4_anon_sbj_111_normalised.nii 파일을 성공적으로 불러왔

In [13]:
label_mapping = {
    'NonDemented': 0,
    'VeryMildDementia': 1,
    'MildDementia': 2,
    'ModerateDementia': 3
}

class AlzheimerDataset(Dataset):
    def __init__(self, df, nii_data_list, target_size=(64, 64, 64), is_train=True):
        self.data = df
        self.nii_data_list = nii_data_list
        self.target_size = target_size
        self.is_train = is_train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # MRI 이미지
        img = self.nii_data_list[idx]
        img = np.array(img, dtype=np.float32)
        img = (img - np.mean(img)) / np.std(img)  # 정규화
        img = torch.tensor(img).unsqueeze(0)  # (D, H, W) -> (1, D, H, W)

        # 크기 조정 (3D 이미지 변환)
        img = F.interpolate(img.unsqueeze(0), size=self.target_size, mode='trilinear', align_corners=False)
        img = img.squeeze(0)  # 배치 차원 제거

        # 텍스트 데이터
        text = self.data.iloc[idx]['CLIP_Text']

        if self.is_train:
            # 레이블 인코딩
            label = label_mapping[self.data.iloc[idx]['Group']]
            return img, text, torch.tensor(label)
        else:
            return img, text

# DataLoader
dataset = AlzheimerDataset(df, nii_data_list, target_size=(64, 64, 64))
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [17]:
from sklearn.model_selection import train_test_split

# 데이터셋 분할
train_df, test_df, train_nii_data_list, test_nii_data_list = train_test_split(
    df, nii_data_list, test_size=0.2, random_state=SEED, stratify=df['Group']
)


# 학습 데이터셋
train_dataset = AlzheimerDataset(train_df, train_nii_data_list, target_size=(64, 64, 64))
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# 검증 데이터셋
test_dataset = AlzheimerDataset(test_df, test_nii_data_list, target_size=(64, 64, 64))
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# 모델

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [19]:
# CLIP 모델 설정
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [20]:
!git clone https://github.com/Tencent/MedicalNet.git
!pip install -r MedicalNet/requirements.txt


Cloning into 'MedicalNet'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 120 (delta 50), reused 43 (delta 43), pack-reused 51 (from 1)
Receiving objects: 100% (120/120), 47.63 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (50/50), done.
ERROR: Could not find a version that satisfies the requirement torch==0.4.1 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==0.4.1


In [21]:
import sys
sys.path.append('/content/MedicalNet')


In [22]:
import argparse
from MedicalNet.model import generate_model

In [25]:
class MedicalNetEncoder(nn.Module):
    def __init__(self, feature_dim=32, use_pretrained=False):
        super(MedicalNetEncoder, self).__init__()
        pretrain_path = "/content/drive/MyDrive/KUBIG/CV 세션 프로젝트/resnet_10.pth" if use_pretrained else None

        opt = argparse.Namespace(
            model='resnet',
            model_depth=10,
            input_W=112,
            input_H=112,
            input_D=16,
            resnet_shortcut='B',
            no_cuda=False,
            n_seg_classes=2,
            gpu_id=[0],
            phase='train',
            pretrain_path=pretrain_path,
            new_layer_names=[]
        )

        self.model, _ = generate_model(opt)
        self.model.fc = nn.Identity()
        self.model.to(device)

    def forward(self, x):
        return self.model(x)

In [26]:
# 모델 정의
class AlzheimerModel(nn.Module):
    def __init__(self, mri_encoder, clip_model):
        super(AlzheimerModel, self).__init__()
        self.mri_encoder = mri_encoder
        self.clip_model = clip_model
        self.fc = nn.Sequential(
            nn.Linear(8704, 512),  # 8704는 결합된 크기에 맞게 설정
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 4)  # 출력 차원을 4로 설정
        )

    def forward(self, image, text):
        image_embeds = self.mri_encoder(image)
        image_embeds = image_embeds.view(image_embeds.size(0), -1)  # (batch_size, -1)

        inputs = clip_processor(text=text, images=None, return_tensors="pt", padding=True).to(device)
        text_embeds = self.clip_model.get_text_features(**inputs)

        combined = torch.cat((image_embeds, text_embeds), dim=1)
        return self.fc(combined)




In [27]:
# 모델 초기화
mri_encoder = MedicalNetEncoder(use_pretrained=True)
model = AlzheimerModel(mri_encoder, clip_model).to(device)

criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류를 위한 손실 함수
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

/content/MedicalNet/models/resnet.py:173: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


loading pretrained model /content/drive/MyDrive/KUBIG/CV 세션 프로젝트/resnet_10.pth


/content/MedicalNet/model.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrain = torch.load(opt.pretrain_path)


# 학습

In [31]:
from sklearn.metrics import accuracy_score, f1_score

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, texts, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images, texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_dataloader)}")

    # 검증
    model.eval()
    test_loss = 0.0
    all_test_preds = []
    all_test_labels = []

    with torch.no_grad():
        for images, texts, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images, texts)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            preds = outputs.argmax(dim=1)
            all_test_preds.extend(preds.cpu().numpy())
            all_test_labels.extend(labels.cpu().numpy())

    test_loss /= len(test_dataloader)
    accuracy = accuracy_score(all_test_labels, all_test_preds)
    f1 = f1_score(all_test_labels, all_test_preds, average='weighted')
    print(f"Validation Loss: {test_loss}, Accuracy: {accuracy}, F1 Score: {f1}")

Epoch [1/10], Loss: 0.7221015564155304
Validation Loss: 0.7285955521193418, Accuracy: 0.7727272727272727, F1 Score: 0.6736596736596737
Epoch [2/10], Loss: 0.7568993537590422
Validation Loss: 0.6932821788571097, Accuracy: 0.7727272727272727, F1 Score: 0.6736596736596737
Epoch [3/10], Loss: 0.7480199660720497
Validation Loss: 0.6831144446676428, Accuracy: 0.7727272727272727, F1 Score: 0.6736596736596737
Epoch [4/10], Loss: 0.724940759525902
Validation Loss: 0.6854462217200886, Accuracy: 0.7727272727272727, F1 Score: 0.6736596736596737
Epoch [5/10], Loss: 0.7168454618632109
Validation Loss: 0.6898470805449919, Accuracy: 0.7727272727272727, F1 Score: 0.6736596736596737
Epoch [6/10], Loss: 0.7356169173686669
Validation Loss: 0.8223356018689546, Accuracy: 0.7727272727272727, F1 Score: 0.6736596736596737
Epoch [7/10], Loss: 0.6669735573660368
Validation Loss: 0.5345371936193921, Accuracy: 0.7727272727272727, F1 Score: 0.6736596736596737
Epoch [8/10], Loss: 0.6953048746974004
Validation Loss: 

In [32]:
# 모델 가중치 확인
for name, param in model.state_dict().items():
    print(f"Layer: {name} | Size: {param.size()}")
    print(f"Values: {param}\n")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        1.3914, 1.5333, 1.4633, 1.5287, 1.4031, 1.5261, 1.5041, 1.5012, 1.3926,
        1.3314, 1.4233, 1.3795, 1.3970, 1.4604, 1.4935, 1.4131, 1.4508, 1.4588,
        1.4783, 1.4467, 1.4212, 1.4764, 1.3905, 1.4914, 1.4899, 1.4760, 1.3647,
        1.3293, 1.5267, 1.5051], device='cuda:0')

Layer: clip_model.vision_model.encoder.layers.8.layer_norm1.bias | Size: torch.Size([768])
Values: tensor([-3.8778e-02,  5.7725e-02,  1.8102e-01, -6.8564e-02,  4.2395e-02,
        -6.7483e-02,  2.0824e-02, -1.1698e-01, -2.2634e-01,  1.1346e-02,
        -7.3195e-02, -5.3841e-03,  9.0748e-03, -2.7329e-02, -1.8558e-01,
        -6.8580e-02,  5.0032e-01, -7.9894e-02, -6.5839e-02, -2.0040e-01,
        -9.9458e-02,  6.7766e-02,  1.1001e-01,  1.7288e-01,  1.2683e-02,
        -5.8448e-02, -2.1187e-01,  5.4149e-02, -8.0545e-02, -9.4354e-02,
        -6.1777e-01,  7.7115e-02,  4.6076e-02,  7.2730e-01,  9.8311e-02,
        -1.1156e-01, -5.5143e-02, -7.0727e-02, -8.2920e-02, -3.

In [33]:
for name, param in model.state_dict().items():
    print(f"Layer: {name} | Size: {param.size()}")

Layer: mri_encoder.model.module.conv1.weight | Size: torch.Size([64, 1, 7, 7, 7])
Layer: mri_encoder.model.module.bn1.weight | Size: torch.Size([64])
Layer: mri_encoder.model.module.bn1.bias | Size: torch.Size([64])
Layer: mri_encoder.model.module.bn1.running_mean | Size: torch.Size([64])
Layer: mri_encoder.model.module.bn1.running_var | Size: torch.Size([64])
Layer: mri_encoder.model.module.bn1.num_batches_tracked | Size: torch.Size([])
Layer: mri_encoder.model.module.layer1.0.conv1.weight | Size: torch.Size([64, 64, 3, 3, 3])
Layer: mri_encoder.model.module.layer1.0.bn1.weight | Size: torch.Size([64])
Layer: mri_encoder.model.module.layer1.0.bn1.bias | Size: torch.Size([64])
Layer: mri_encoder.model.module.layer1.0.bn1.running_mean | Size: torch.Size([64])
Layer: mri_encoder.model.module.layer1.0.bn1.running_var | Size: torch.Size([64])
Layer: mri_encoder.model.module.layer1.0.bn1.num_batches_tracked | Size: torch.Size([])
Layer: mri_encoder.model.module.layer1.0.conv2.weight | Size:

# 평가

In [ ]:
# from sklearn.metrics import accuracy_score, roc_auc_score

# model.eval()
# all_preds = []
# all_labels = []

# with torch.no_grad():
#     for images, texts, labels in dataloader:
#         images = images.to(device)
#         labels = labels.to(device)

#         outputs = model(images, texts)
#         preds = (outputs.squeeze() > 0.5).int()

#         all_preds.extend(preds.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())

# accuracy = accuracy_score(all_labels, all_preds)
# roc_auc = roc_auc_score(all_labels, all_preds)
# print(f"Accuracy: {accuracy}, AUC: {roc_auc}")